###
Extract Data 4hour  

In [ ]:
#library
library(tidyverse)
#set directory for  tables
sqldir = '../sqltables/'
outdir = '../processed_mimic/'

In [ ]:
##get cohort to define time interval 
cohort_df <- read_csv(paste0(sqldir, 'cohort.csv'), col_types=cols()) %>% select(-X1)
dim(cohort_df)
head(cohort_df,2)
timew = '4 hour'
########################
# DATA TRANSFORMATIONS
get_time_intervals <- function(start, end) {
  interval_times <- seq(start, end, by = timew)
  interval_start_time <- interval_times[1:length(interval_times) -1]
  interval_end_time <- interval_times[2:length(interval_times)]
  return(data.frame(interval_start_time, interval_end_time))
}
interval_times_df <- cohort_df %>% 
  group_by(subject_id, hadm_id, icustay_id) %>% 
  do(get_time_intervals(.$window_start, .$window_end))

# Visual check
dim(interval_times_df)

### Processing Vassopressor 

In [ ]:
# Load some vassopressors
vassopressor_cv_df <- read_csv(paste0(sqldir, 'vassopressors_cv_cohort.csv'),
                              col_types=cols())
vassopressor_mv_df <- read_csv(paste0(sqldir, 'vassopressors_mv_cohort.csv'),
                              col_types=cols())

#load cv vasso
max_vassopressor_cv <- interval_times_df %>% 
  left_join(vassopressor_cv_df) %>% 
  group_by(subject_id, 
           hadm_id, 
           icustay_id, 
           interval_start_time, 
           interval_end_time) %>%
  filter(charttime > interval_start_time & charttime <= interval_end_time) %>%
  summarise(max_amount = max(mcgkgmin),
  median_vaso_dose = median(mcgkgmin)) %>% mutate(max_amount = case_when(max_amount > 1e-02 ~ max_amount, TRUE ~ 0))

 #load mv vasso
max_vassopressor_mv <- interval_times_df %>% 
  left_join(vassopressor_mv_df) %>% 
  group_by(subject_id, 
           hadm_id, 
           icustay_id, 
           interval_start_time, 
           interval_end_time) %>%
  filter(endtime > interval_start_time & endtime <= interval_end_time) %>%
  summarise(max_amount = max(mcgkgmin),
  median_vaso_dose = median(mcgkgmin) ) %>% mutate(max_amount = case_when(max_amount > 1e-02 ~ max_amount, TRUE ~ 0)) 
  

In [ ]:
# A function to map the data to quartiles
num2quartile <- function(x) {
  #     x <- x[is.na(x)]
  quants <- quantile(x, probs=0:4/4, na.rm = TRUE)
  print(round(quants,5))
  return(as.integer(cut(x, quants, include.lowest=TRUE)))
}
MIMIC_MV_VP_Quantiles = num2quartile(max_vassopressor_mv$max_amount)

# Map to discrete vassopressor states
temp_VP <- max_vassopressor_cv %>% 
  full_join(max_vassopressor_mv) %>%
  filter(!is.na(max_amount)) %>% 
  filter(!is.na(median_vaso_dose))

In [ ]:
vpbreak = c(-Inf,0, 0.110, 0.225, 0.450, Inf) # 4hourssetting

# merge vasopressor with interval_times (essentialy adds empty interval times that were left out in the initial max_vasopressor_mv join)
action_df_VP <- temp_VP %>% right_join(interval_times_df) %>% group_by(subject_id, hadm_id, icustay_id, interval_start_time, interval_end_time) %>% summarise(max_amount = max(max_amount), median_vaso_dose = median(median_vaso_dose)) 

### add action MIMIC
action_df_VP$discrete_VP <- as.integer(cut(action_df_VP$max_amount, breaks = vpbreak))-1 

# Change NA to zero's
action_df_VP$max_amount[is.na(action_df_VP$max_amount)] <- 0
action_df_VP$discrete_VP[is.na(action_df_VP$discrete_VP)] <- 0
action_df_VP$median_vaso_dose[is.na(action_df_VP$median_vaso_dose)] <- 0


# Visual inspection
head(action_df_VP,2)
nrow(action_df_VP)
table(action_df_VP$discrete_VP)

### Process fluids

In [ ]:
#processing fluids 
########################
# Pulling out some data.table stuff to go FAST
library(data.table)
interval_times_dt <- as.data.table(interval_times_df, key = 'icustay_id')
head(interval_times_dt,2)
##import carevue fluids 
########################
# load CAREVUE FLUIDS
inputevents_cv_df <- read_csv(paste0(sqldir, 'inputevents_cv_cohort.csv'),
                             col_types=cols())
ie_filt_cv <- inputevents_cv_df %>% filter(amountuom == 'ml' & !is.na(amount) & amount >= 0 & amount <= 5000)
ie_filt_cv_dt <- as.data.table(ie_filt_cv, key = 'icustay_id')

# preprocess CAREVUE FLUIDS
total_IV_cv <- interval_times_dt %>% 
  merge(ie_filt_cv_dt, allow.cartesian = TRUE) %>% 
  group_by(subject_id,
           hadm_id, 
           icustay_id, 
           interval_start_time, 
           interval_end_time) %>%
  filter(charttime > interval_start_time & charttime <= interval_end_time) %>%
  summarise(total_amount = sum(amount))

  # Load the MV fluids
# Load the fluids
inputevents_mv_df <- read_csv(paste0(sqldir, 'inputevents_mv_cohort.csv'))


MV_input = inputevents_mv_df %>% select(subject_id, icustay_id, hadm_id, starttime, endtime, 
                                        amount, amountuom, rate, rateuom, orderid, ordercategoryname, 
                                        secondaryordercategoryname, patientweight, totalamount) %>% 
                                 filter(amountuom == 'ml' & ordercategoryname != '14-Oral/Gastric Intake' & !is.na(amount) & amount > 0 & amount < 5000) %>% 
                                 mutate(duration = as.numeric(c(difftime(endtime, starttime)))) %>% 
                                 mutate(amount = round(amount,2)) %>% 
                                 select(-rate, -rateuom,-orderid,-ordercategoryname,-secondaryordercategoryname,-patientweight, -amountuom,-totalamount)
head(MV_input)

 ### Processing metavision fluids

### 1. METAVISION IV dose started before interval and ended after interval

In [ ]:
total_IV_during <- interval_times_dt %>% left_join(MV_input, allow.cartesian = TRUE) %>% group_by(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time) %>%
                   filter(starttime < interval_start_time & endtime > interval_end_time) 

all_IV_during <- total_IV_during %>% mutate(total_amount_during = as.numeric(interval_end_time-interval_start_time) * (amount/duration))

all_IV_during <- all_IV_during %>% select(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time, total_amount_during) %>% 
                                   group_by(subject_id, icustay_id, hadm_id, interval_start_time ,interval_end_time) %>% 
                                   summarise(total_amount = sum(total_amount_during))


### 2. METAVISION IV dose started before and ended within interval

In [ ]:
# IV dose started before interval and ended within interval
total_IV_before <- interval_times_dt %>% left_join(MV_input, allow.cartesian = TRUE) %>% group_by(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time) %>%
                   filter(starttime < interval_start_time & endtime > interval_start_time & endtime <= interval_end_time) 

all_IV_before <- total_IV_before %>% mutate(total_amount_before = as.numeric(difftime(endtime, interval_start_time, units='mins')) * (amount/duration))

all_IV_before <- all_IV_before %>% select(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time, total_amount_before) %>% 
                                   group_by(subject_id, icustay_id, hadm_id, interval_start_time ,interval_end_time) %>% 
                                   summarise(total_amount = sum(total_amount_before))


### 3. METAVISION IV dose started within interval and ended within interval

In [ ]:
# IV dose started within the interval and ended within the interval
all_IV_within <- interval_times_dt %>% left_join(MV_input, allow.cartesian = TRUE) %>% group_by(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time) %>%
                 filter(starttime >= interval_start_time & starttime < interval_end_time & endtime <= interval_end_time & endtime > interval_start_time) %>%
                 summarise(total_amount = sum(amount))

### 4. METAVISION IV Dose started within interval and ended after interval

In [ ]:
# IV dose started within interval and ended after interval
total_IV_after <- interval_times_dt %>% left_join(MV_input, allow.cartesian = TRUE) %>% group_by(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time) %>%
                   filter(starttime > interval_start_time & starttime < interval_end_time & endtime > interval_end_time) 

all_IV_after <- total_IV_after %>% mutate(total_amount_after = round((as.numeric(difftime(interval_end_time, starttime, units='mins'))/duration)*(amount/duration),3))

all_IV_after <- all_IV_after %>% select(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time, total_amount_after) %>% 
                                 group_by(subject_id, icustay_id, hadm_id, interval_start_time,interval_end_time) %>% 
                                 summarise(total_amount = sum(total_amount_after))

In [ ]:
summary(all_IV_before$total_amount)
summary(all_IV_within$total_amount)
summary(all_IV_after$total_amount)
summary(all_IV_during$total_amount)

### 5. Join all METAVISION IV tables

In [ ]:
# full join all IV tables
total_IV <- all_IV_during %>% full_join(all_IV_before) %>% 
                                full_join(all_IV_within) %>% 
                                full_join(all_IV_after) %>% 
                                group_by(subject_id, icustay_id, hadm_id, interval_start_time, interval_end_time) %>%
                                summarise(total_amount = sum(total_amount))



#### 6. Create QUARTILES actions for the IV fluid dose BASED ON MV ONLY!!! as these are more consistent

In [ ]:
#create quartile 
MV_IV_Fluids = num2quartile(total_IV$total_amount)


#### Merge Metavision and CareVue fluids

In [ ]:
# Map to discrete IV states
temp_IV <- total_IV_cv %>% 
  full_join(total_IV) %>%
  filter(!is.na(total_amount))

# merge vasopressor with interval_times (essentialy adds empty interval times that were left out in the initial max_vasopressor_mv join)
action_df_IV <- temp_IV %>% right_join(interval_times_df) %>% 
                group_by(subject_id, hadm_id, icustay_id, interval_start_time, interval_end_time) %>% 
                summarise(total_amount = sum(total_amount)) 


In [ ]:
### add action MIMIC
# this is for 4 hours
#ivbreak = c(-Inf,0.0000,   100.0000 ,  291.8409 ,  752.4806, Inf) # 4hour setting include gastro
 ivbreak = c(-Inf,0 ,   50 ,  217 ,  700, Inf) 


action_df_IV$discrete_IV <- as.integer(cut(action_df_IV$total_amount, breaks = ivbreak))-1 

# Change NA to zero's
action_df_IV$total_amount[is.na(action_df_IV$total_amount)] <- 0
action_df_IV$discrete_IV[is.na(action_df_IV$discrete_IV)] <- 0

# Visual inspection
head(action_df_IV,2)
nrow(action_df_IV)

action_df <- action_df_IV %>% full_join(action_df_VP)
# Get the final state as discrete_IV + 5 * discrete_pressor
action_df <- action_df %>% mutate(discrete_action = discrete_IV + 5 * discrete_VP)
action_df <- action_df %>% rename(total_IV = total_amount, max_VP = max_amount)
head(action_df)
table(action_df$discrete_action)

In [ ]:
#add cummulative IV 
# Add running total
action_df <- action_df %>% group_by(icustay_id) %>% 
                arrange(subject_id,hadm_id,icustay_id,interval_start_time, interval_end_time) %>% 
                mutate(Running_total_IV = cumsum(total_IV))
# check first:
table(action_df_IV$discrete_IV)
table(action_df_VP$discrete_VP)

In [ ]:
#write action output ( the quartile change if  used gastro intake)
write_csv(action_df, paste0(outdir, 'action_df.csv')) 
write_csv(interval_times_df, paste0(outdir, 'interval_times_df.csv'))

In [ ]:
dim(interval_times_df)
head(interval_times_df,2)
dim(action_df)
head(action_df,2)

### Process Vitals

In [ ]:

### import vitals data
vitals_df <- read_csv(paste0(sqldir, 'vitals_cohort.csv'), 
                      col_types=cols())

# Cast to DataTable
vitals_dt <- as.data.table(vitals_df, key = 'icustay_id')

#read intervals 
interval_times_df = read_csv(paste0(outdir, 'interval_times_df.csv'), 
                      col_types=cols())

interval_times_dt <- as.data.table(interval_times_df, key = 'icustay_id')

# Assign vital signs to the corresponding interval times
merged_vitals <- interval_times_dt %>% 
                    merge(vitals_dt, allow.cartesian = TRUE)

# take the mean vital sign for each interval time
mean_vitals <- merged_vitals[charttime > interval_start_time & charttime <= interval_end_time,
                                   .(mean_vital = mean(valuenum)), by = .(subject_id, 
                                                                          hadm_id, 
                                                                          icustay_id, 
                                                                          interval_start_time, 
                                                                          interval_end_time, 
                                                                          vital_id)]
# visual check merged vitals
head(mean_vitals,2)



### Process Labs

In [ ]:
########################
# load the labs data
labs_dt <- read_csv(paste0(sqldir, 'labs_cohort.csv'), 
                    col_types=cols())
labs_dt <- as.data.table(labs_dt, key = 'icustay_id')

# visual check
head(labs_dt,2)

# Assign lab value to the corresponding interval times
merged_labs <- interval_times_dt %>% 
                    merge(labs_dt, allow.cartesian = TRUE)

# take the mean lab value for each interval time
mean_labs <- merged_labs[charttime > interval_start_time & charttime <= interval_end_time,
                                   .(mean_lab = mean(valuenum)), by = .(subject_id, 
                                                                          hadm_id, 
                                                                          icustay_id, 
                                                                          interval_start_time, 
                                                                          interval_end_time, 
                                                                          lab_id)]
# visual check merged labs
head(mean_labs,2)

### Merge Labs and Vitals

In [ ]:
########################
### Merge Labs and Vitals

# make it pretty
mean_labs <- mean_labs %>% rename(meas_id = lab_id, mean_value = mean_lab)
mean_vitals <- mean_vitals %>% rename(meas_id = vital_id, mean_value = mean_vital)

# Join tables
mean_labs_vitals <- mean_labs %>% full_join(mean_vitals)
vitals_labs_spread <- mean_labs_vitals %>% 
                        spread(meas_id, mean_value) %>% 
                        right_join(interval_times_dt) 

# Exclude times in which no measurements were made
vitals_labs_spread_filt <- vitals_labs_spread %>%
                                gather(lab_id, meas_value, ALBUMIN:WBC) %>%
                                group_by(subject_id, hadm_id, icustay_id, interval_start_time, interval_end_time) %>%
                                summarise(exclude = all(is.na(meas_value))) %>%
                                full_join(vitals_labs_spread) %>%
                                filter(!exclude)

# visual check of combined vitals and labs spread file (final file)
head(vitals_labs_spread_filt,10)

In [ ]:
########################
### export data to csv
write_csv(vitals_labs_spread_filt, paste0(outdir, 'vitals_labs_spread_filt.csv'))

In [ ]:
dim(vitals_labs_spread_filt)

In [ ]:
# calculate the means of each combination of 'grade' and 'sex' with `group_by`
means_lab_measure <- mean_labs %>% group_by(meas_id) %>% 
   summarise(mean = mean(mean_value))

# making the plot
ggplot(means_lab_measure, aes(x = meas_id, y = mean)) +
    geom_bar(stat = "identity", position = "dodge")

In [ ]:
# calculate the means of each combination of 'grade' and 'sex' with `group_by`
means_vital_measure <- mean_vitals %>% group_by(meas_id) %>% 
   summarise(mean = mean(mean_value))

# making the plot
ggplot(means_vital_measure, aes(x = meas_id, y = mean)) +
    geom_bar(stat = "identity", position = "dodge")

### Combining Vitals + Lab + FIO2 + UrineOutput and Demographics 

In [ ]:
### IMPORT
vitals_labs <- read_csv(paste0(outdir, 'vitals_labs_spread_filt.csv'),
                       col_types=cols())
demographics <- read_csv(paste0(sqldir, 'demographics_cohort.csv'),
                        col_types=cols())
urine_df <- read_csv(paste0(sqldir, 'UrineOutput_cohort.csv'),
                    col_types=cols())
interval_times_dt =  read_csv(paste0(outdir, 'interval_times_df.csv'),
                       col_types=cols())  
interval_times_dt <- as.data.table(interval_times_dt, key = 'icustay_id')
                

In [ ]:
# forward fill vital labs measurements 
# Perform LVCF
vitals_labs_lvcf <- vitals_labs %>% group_by(subject_id, hadm_id, icustay_id) %>%
                        arrange(subject_id, hadm_id, icustay_id, interval_start_time) %>%
                        fill(ALAT:WBC) %>% select(-'<NA>')# ALAT:WBC is based on the on 7th up to 40th collumn corresponding to all the actual features in this dataframe
dim(vitals_labs_lvcf)

## get urine + cummulative Urine 


In [ ]:
### get urine data  and calculate cummulative 
# load the urine output data
urine_df <- read_csv(paste0(sqldir, 'UrineOutput_cohort.csv'),
                    col_types=cols())
urine_dt <- as.data.table(urine_df, key = 'icustay_id') %>% filter(value<5000 & value>0)

# Merge with interval times
merged_urine <- interval_times_dt %>% merge(urine_dt, allow.cartesian = TRUE)

# MERGE WITH MEAN() + Keep only 'real value' (urine production within interval times)
total_urine_df <- merged_urine[charttime > interval_start_time & charttime <= interval_end_time,
                         .(total_UP = sum(value)), by = .(subject_id,hadm_id,icustay_id, 
                                                           interval_start_time, 
                                                           interval_end_time)]

### Get FiO2 and forward fill

In [ ]:
### get FiO2
# load the urine output data
fio2_dt <- read_csv(paste0(sqldir, 'FiO2_cohort.csv'),
                   col_types=cols())
fio2_dt <- as.data.table(fio2_dt, key = 'icustay_id')

# Merge with interval times
merged_fio2 <- interval_times_dt %>% merge(fio2_dt, allow.cartesian = TRUE)

# MERGE WITH MEAN() + Keep only 'real value' (urine production within interval times)
total_fio2_df <- merged_fio2[charttime > interval_start_time & charttime <= interval_end_time,
                         .(FiO2 = mean(fio2)), by = .(subject_id,hadm_id,icustay_id, 
                                                           interval_start_time, 
                                                           interval_end_time)]


In [ ]:
total_fio2_df <- total_fio2_df %>% right_join(interval_times_dt) %>%  group_by(subject_id,hadm_id,
                                                        icustay_id,interval_start_time, interval_end_time)

total_fio2_df <- total_fio2_df %>% group_by(subject_id, hadm_id, icustay_id) %>%
                        arrange(subject_id, hadm_id, icustay_id, interval_start_time) %>%
                        fill(FiO2)

# visual check
dim(total_fio2_df)
dim(interval_times_dt)

In [ ]:
head(total_fio2_df, 18)

#fill missing FiO2 with mean 
total_fio2_df <- total_fio2_df %>% group_by(icustay_id) %>%
                mutate(FiO2=ifelse(is.na(FiO2),mean(FiO2,na.rm=TRUE),FiO2))
## remove where total fio2 was still missing as those patients does not have records
total_fio2_df = total_fio2_df[!is.na(total_fio2_df$FiO2), ]
dim(total_fio2_df)

### Join FiO2 with Vitals 

In [ ]:
vitals_labs_fio2  = vitals_labs_lvcf %>% left_join(total_fio2_df) %>% ungroup()
dim(vitals_labs_fio2)
length(table(vitals_labs_fio2$icustay_id))
head(vitals_labs_fio2,18)

### Join Urine with FiO2 + Vitals 

In [ ]:
vitals_labs_fio2_up  = vitals_labs_fio2 %>% left_join(total_urine_df) %>% ungroup()
dim(vitals_labs_fio2_up)
length(table(vitals_labs_fio2_up$icustay_id))
#### first imppute 
vitals_labs_fio2_up = vitals_labs_fio2_up %>% group_by(subject_id,hadm_id,icustay_id) %>%replace_na(list(total_UP = 0))
# add running total
vitals_labs_fio2_up <- vitals_labs_fio2_up %>% 
group_by(subject_id,hadm_id,icustay_id) %>% 
arrange(subject_id,hadm_id,icustay_id, interval_start_time, interval_end_time) %>%
mutate(Running_total_UP = cumsum(total_UP))

In [ ]:
head(vitals_labs_fio2_up, 6)

### Join Vital + Lab + Urine + FiO2  with Demographics

In [ ]:
vitals_labs_fio2_up_demo = vitals_labs_fio2_up  %>% 
                        left_join(demographics) %>% 
                        ungroup()
dim(vitals_labs_fio2_up_demo)

In [ ]:
head(vitals_labs_fio2_up_demo)

In [ ]:
write_csv(vitals_labs_fio2_up_demo, paste0(outdir, 'vitals_labs_fio2_up_demo.csv'))

In [ ]:
# IMPORT DATA
action_df <- read_csv(paste0(outdir, 'action_df.csv'),
                      col_types=cols())
cohort_df <- read_csv(paste0(sqldir, 'cohort.csv'),
                      col_types=cols())
alldata_df = vitals_labs_fio2_up_demo

########################
### Merge all data
outcome_df <- cohort_df %>% group_by(subject_id, 
                                     hadm_id, 
                                     icustay_id)

########################
### Combine vitals and labs (data_all.csv) with actions (action_df.csv) and cohort (cohort.csv: only for discharge[0=alive])
alldata_df_action <- alldata_df %>% inner_join(select(action_df, subject_id, 
                                                      hadm_id, 
                                                      icustay_id, 
                                                      interval_start_time, 
                                                      interval_end_time,
                                                      discrete_action,
                                                      total_IV,
                                                      Running_total_IV,
                                                      max_VP,
                                                      median_vaso_dose,
                                                      discrete_IV, discrete_VP)) %>%
  inner_join(select(cohort_df, subject_id, hadm_id, 
  icustay_id, hospital_expire_flag ))

In [ ]:
head(alldata_df_action)

### Add sparse rewards

In [ ]:
## add reward to the data 
########################
### Add final reward and zeros for intermediate reward
temp_alldata <- alldata_df_action %>% group_by(subject_id, 
                                               hadm_id, 
                                               icustay_id,
                                               hospital_expire_flag) %>%
  summarise(interval_start_time = max(interval_start_time)) %>%
  mutate(reward = ifelse(hospital_expire_flag, -15, 15))

# add "intermediate reward" to dataset (all 'NA' rewards are set to 0)
temp_alldata_action_reward <- alldata_df_action %>% 
  left_join(temp_alldata) %>%
  replace_na(list(reward = 0))

### for time series total_IV and max_VP Action shift (lag one state behind current state)

In [ ]:
final_df_all = temp_alldata_action_reward %>% 
        group_by(icustay_id) %>% 
        mutate(total_IV_prev = lag(total_IV,order_by=icustay_id)) %>% 
        mutate(total_IV_prev = replace_na(total_IV_prev, 0)) %>%
        ungroup()

final_df_all = temp_alldata_action_reward %>% 
        group_by(icustay_id) %>%
        mutate(max_VP_prev = lag(max_VP,order_by=icustay_id)) %>% 
        mutate(max_VP_prev = replace_na(max_VP_prev, 0)) %>%
        ungroup()

#### Rename some of the columns 

In [ ]:
MIMIC_all = final_df_all %>%  rename(discrete_action = discrete_action) %>%             # MIMIC action   to ICV action   (no conversion needed) 
                                rename(max_VP = max_VP) %>%                               # MIMIC action   to ICV action   (no conversion needed) 
                                rename(total_IV = total_IV) %>%                           # MIMIC action   to ICV action   (no conversion needed) 
                                rename(Discharge = hospital_expire_flag) %>%              # MIMIC 1=death  to ICV 1=death  (no conversion needed) 
                                rename(Reward = reward) %>%                               # MIMIC score    to ICV score    (no conversion needed) 
                                rename(Sirs_score = sirs) %>%                             # MIMIC score    to ICV score    | ICV admission | MIMIC -> sepsis3.sql (onset)
                                rename(Sofa_score = sofa) %>%                             # MIMIC score    to ICV score    | ICV admission | MIMIC -> sepsis3.sql (onset)
                                rename(Weight = weight) %>%                               # MIMIC kg       to ICV kg       | ICV admission | MIMIC -> sepsis3.sql (onset)
                                rename(Ventilator = vent) %>%                             # MIMIC 0/1      to ICV 0/1      | (-2 to +24h)  | MIMIC -> sepsis3.sql (-4 to +24h)
                                rename(Height = height) %>%                               # MIMIC cm       to ICV cm       (no conversion needed)
                                rename(Age = age) %>%                                     # MIMIC years    to ICV years    (no conversion needed)
                                rename(Gender = is_male) %>%                              # MIMIC male=1   to ICV male=1   (no conversion needed)
                                rename(HeartRate = HeartRate) %>%                         # MIMIC /min     to ICV /min     (no conversion needed) 
                                rename(Temp = TempC) %>%                                  # MIMIC Celcius  to ICV Celcius  (no conversion needed) 
                                rename(MAP = MeanBP) %>%                                  # MIMIC mmHg     to ICV mmHg     (no conversion needed) 
                                rename(DIA = DiasBP) %>%                                  # MIMIC mmHg     to ICV mmHg     (no conversion needed) 
                                rename(SYS = SysBP) %>%                                   # MIMIC mmHg     to ICV mmHg     (no conversion needed) 
                                rename(RespRate = RespRate) %>%                           # MIMIC /min     to ICV /min     (no conversion needed) 
                                rename(SpO2 = SpO2) %>%                                   # MIMIC %        to ICV %        (no conversion needed) 
                                rename(Natrium = SODIUM) %>%                              # MIMIC mEq/L    to ICV mmol/L   (no conversion needed) 
                                rename(Chloride = CHLORIDE) %>%                           # MIMIC mEq/L    to ICV mmol/L   (no conversion needed)
                                rename(Kalium = POTASSIUM) %>%                            # MIMIC mEq/L    to ICV mmol/L   (no conversion needed) 
                                rename(Trombo = PLATELET) %>%                             # MIMIC k/ul     to ICV 10^9/L   (no conversion needed) 
                                rename(LEU = WBC) %>%                                     # MIMIC K/uL     to ICV 10e^9/L  (no conversion needed)
                                rename(ANION_GAP = 'ANION GAP') %>%                       # MIMIC mEq/L    to ICV mmol/l   (no conversion needed)
                                rename(APTT = PTT) %>%                                    # MIMIC sec      to ICV sec      (no conversion needed)
                                rename(Art_PH = PH) %>%                                   # MIMIC geen     to ICV geen     (no conversion needed)
                                rename(ASAT = ASAT) %>%                                   # MIMIC IU/L     to ICV IE/L     (no conversion needed)
                                rename(ALAT = ALAT) %>%                                   # MIMIC IU/L     to ICV IE/L     (no conversion needed)
                                rename(Bicarbonaat = BICARBONATE) %>%                     # MIMIC mEq/L    to ICV mmol/L   (no conversion needed)
                                rename(Art_BE = BaseExcess) %>%                           # MIMIC mEq/L    to ICV mmol/L   (no conversion needed)
                                rename(Ion_Ca = ION_CALCIUM) %>%                          # MIMIC mmol/l   to ICV mmol/L   (no conversion needed)
                                rename(Lactate = LACTATE) %>%                             # MIMIC ?        to ICV ?        (no conversion needed)
                                rename(PaCO2 = PACO2) %>%                                 # MIMIC mmHg     to ICV mmhg     (ICV SQL file converts Kpa to mmHg values, no further conversion needed)
                                rename(PaO2 = PAO2) %>%                                   # MIMIC mmHg     to ICV mmhg     (ICV SQL file converts Kpa to mmHg values, no further conversion needed)
                                mutate(Shock_Index = HeartRate / SYS) %>%                 # MIMIC score    to ICV score    (calculation of score) 
                                mutate(HB = HEMOGLOBIN * 0.6206) %>%                      # MIMIC g/dL     to ICV mmol/l   (http://www.scymed.com/en/smnxpf/pfxdq210_c.htm)
                                mutate(Bili = BILIRUBIN * (1/0.05847953)) %>%             # MIMIC mg/dl    to ICV umol/L   (http://www.endmemo.com/medical/unitconvert/Bilirubin.php)
                                mutate(Creat = CREATININE * (1/0.01131222)) %>%           # MIMIC mg/dl    to ICV umol/l   (http://www.endmemo.com/medical/unitconvert/Creatinine.php)
                                mutate(INR = PT/12) %>%                                   # MIMIC sec      to ICV INR      (normal_PT~12)
                                mutate(Ureum = BUN * 0.3571) %>%                          # MIMIC mg/dL    to ICV mmol/L   (http://www.scymed.com/en/smnxps/psxff047_c.htm)
                                mutate(Albumine = ALBUMIN * 10) %>%                       # MIMIC g/dL     to ICV g/L      (simple conversion)
                                mutate(Magnesium = MAGNESIUM * 0.411) %>%                 # MIMIC mg/dl    to ICV mmol/l   (http://www.endmemo.com/medical/unitconvert/Magnesium.php)
                                mutate(Calcium = CALCIUM * 0.25) %>%                      # MIMIC mg/dl    to ICV mmol/L   (http://www.endmemo.com/medical/unitconvert/Calcium.php)
                                mutate(PF_ratio = PaO2 / FiO2) %>%
                                rename(glucose = Glucose) %>% 
                                       mutate(glutemp = rowMeans(data.frame(GLUCOSE, glucose),na.rm = TRUE)) %>% 
                                       mutate(Glucose = glutemp * 0.0555) %>%             # MIMIC mg/dL    to ICV mmol/l   (http://www.endmemo.com/medical/unitconvert/Glucose.php)
                                select(-ALBUMIN, -CALCIUM, - CREATININE, -glucose, 
                                       -GLUCOSE, -glutemp, -BUN, -MAGNESIUM, 
                                       -HEMOGLOBIN, -BILIRUBIN,- PT)   

In [ ]:
dim(MIMIC_all)

In [ ]:
head(MIMIC_all)

In [ ]:
MIMIC_all = MIMIC_all[,c(names(MIMIC_all)[1:5],sort(colnames(MIMIC_all[6:ncol(MIMIC_all)] )))]
print(names(MIMIC_all))
head(MIMIC_all,5)

### Add information related to datasource

In [ ]:
sepsis_df <- read_csv(paste0(sqldir, 'sepsis3-df-no-exclusions.csv'),
                     col_types=cols()) %>% select(icustay_id, dbsource, composite_outcome)

In [ ]:
### Combine data with admission source
MIMIC_all <- MIMIC_all %>% left_join(sepsis_df)
summary(as.factor(MIMIC_all$dbsource))

In [ ]:
head(MIMIC_all)

In [ ]:
#total patients
length(table(MIMIC_all$icustay_id))

In [ ]:
########################
### export data to csv ( This is Raw file and contains missing values)
write_csv(MIMIC_all, paste0(outdir, 'MIMIC_data.csv'))   ### Most Important file

###  Preprocessing For Metavision Data Only

In [ ]:
MIMIC_MV = MIMIC_all %>% filter(dbsource == 'metavision') %>% select(-dbsource)
head(MIMIC_MV,1)
dim(MIMIC_MV)

########################
### export data to csv
write_csv(MIMIC_MV, paste0(outdir, 'MIMIC_MVdata.csv'))

In [ ]:
colnames(MIMIC_MV)

### Visual Checks

In [ ]:
## Required packages
library(ggplot2)
library(RColorBrewer)
library(colorRamps)
## You need to expand palette size
colourCount = 21 # number of actions
getPalette = colorRampPalette(brewer.pal(9, "Set2"))
long_colors = colorRampPalette(brewer.pal(12, "Spectral"))(colourCount)

# plot
ggplot( MIMIC_MV, aes(total_IV, max_VP, color = as.factor(discrete_action))) + 
        geom_point() + scale_color_manual(values = long_colors) + 
        scale_y_log10(limits = c(0.01,10)) + 
        scale_x_log10(limits = c(0.01,10000), labels = scales::comma) +
        ggtitle("Fluids (IV) vs Vasopressor dose Quartile bins") + 
        theme(legend.position="none") + 
        labs(subtitle="MIMIC Metavision dataset",y = "Max VP Dose (ugram/min)",x = "Total IV fluid (mL)",caption="X and Y Axis on log10 scale") 

#### Getting Sick Patients based on Sofa

In [ ]:
# Apply IV fluid filter
sick_MIMIC <- MIMIC_MV %>% group_by(icustay_id) %>%
                        filter(interval_start_time == max(interval_start_time)) %>%
                        summarise(END_total_IV = max(Running_total_IV)) %>%
                        filter(END_total_IV > 100) %>%
                        filter(END_total_IV < 40000)

# Apply SOFA filter
sofa_MIMIC <- MIMIC_MV %>% group_by(icustay_id) %>%
                        summarise(SOFA = mean(Sofa_score, na.rm = TRUE)) %>%
                        filter(SOFA>3)

# Join filters
filt_MIMIC = sofa_MIMIC # sick_MIMIC %>% inner_join(sofa_MIMIC) %>% group_by(PatientID)

# inspect filtered dataset
length(unique(MIMIC_MV$icustay_id))
nrow(filt_MIMIC)
summary(filt_MIMIC)

# apply filters and create new dataset from subset of MIMIC data 
filt_MIMIC_dataset <- MIMIC_MV %>% filter (icustay_id %in% c(filt_MIMIC$icustay_id))

########################
### export data to csv
write_csv(filt_MIMIC_dataset, paste0(outdir, 'MIMIC_MVsick_data.csv'))
#this is mimic file of sick patients

In [ ]:
## Required packages
library(ggplot2)
library(RColorBrewer)
library(colorRamps)


## You need to expand palette size
colourCount = 21 # number of actions
getPalette = colorRampPalette(brewer.pal(9, "Set2"))
long_colors = colorRampPalette(brewer.pal(12, "Spectral"))(colourCount)

# plot
ggplot( filt_MIMIC_dataset, aes(total_IV, max_VP, color = as.factor(discrete_action))) + 
        geom_point() + scale_color_manual(values = long_colors) + 
        scale_y_log10(limits = c(0.01,10)) + 
        scale_x_log10(limits = c(0.01,10000), labels = scales::comma) +
        ggtitle("Fluids (IV) vs Vasopressor dose Quartile bins") + 
        theme(legend.position="none") + 
        labs(subtitle="MIMIC Metavision SOFA>3 dataset",y = "Max VP Dose (ugram/min)",x = "Total IV fluid (mL)",caption="X and Y Axis on log10 scale") 

In [ ]:
cat('Analysis Finished for time window', outdir)


In [ ]:
colnames(MIMIC_all)

#### Data Cleaning Part From all Samples ( Carevyue + MIMIC)

In [ ]:
library(tidyverse)
library(data.table)
datadir = '../processed_data/fourhour/'


In [ ]:
MIMIC_all = read_csv(paste0(datadir, 'MIMIC_data.csv'),  col_types = cols())

### I noticed there is no total IV prevcreated 
final_df_all = temp_alldata_action_reward %>% 
        group_by(icustay_id) %>% 
        mutate(total_IV_prev = lag(total_IV,order_by=icustay_id)) %>% 
        mutate(total_IV_prev = replace_na(total_IV_prev, 0)) %>%
        ungroup()

In [ ]:
#add total_IV_prev to the data 
MIMIC_all= MIMIC_all %>% 
        group_by(icustay_id) %>% 
        mutate(total_IV_prev = lag(total_IV,order_by=icustay_id)) %>% 
        mutate(total_IV_prev = replace_na(total_IV_prev, 0)) %>%
        ungroup()

In [ ]:
colnames((MIMIC_all))

In [ ]:
write_csv(MIMIC_all, paste0(datadir, 'MIMIC_data.csv')) 

## This is missing in MV as well so rewrite MV 

In [ ]:
library(tidyverse)
datadir = '../processed_data/fourhour/'
MIMIC_all =  read_csv(paste0(datadir, 'MIMIC_data.csv'), col_types=cols())

In [ ]:
MIMIC_MV = MIMIC_all %>% filter(dbsource == 'metavision') %>% select(-dbsource)
head(MIMIC_MV,1)
dim(MIMIC_MV)
colnames(MIMIC_MV)

########################
### export data to csv
write_csv(MIMIC_MV, paste0(datadir, 'MIMIC_MVdata.csv'))